In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

In [3]:
spacex_df=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=8)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=5000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(30,30),
        icon_anchor=(0,0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`

*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map

In [7]:
for _, site in launch_sites_df.iterrows():
    
    # Circle to show launch site area
    folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=2000,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.4
    ).add_to(site_map)
    
    # Marker for launch site name
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=folium.Popup(site['Launch Site'], max_width=250),
        tooltip=site['Launch Site'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)


In [8]:
site_map

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

In [10]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value

In [11]:
spacex_df['marker_color'] = spacex_df['class'].map({1: 'green', 0: 'red'})

In [12]:
marker_cluster = MarkerCluster().add_to(site_map)

for _, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color'], icon='info-sign'),
        outcome = 'Success' if record['class'] == 1 else 'Failure',
        tooltip=f"{record['Launch Site']} | class={record['class']}"
    ).add_to(marker_cluster)


In [13]:
site_map

# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [16]:
# CCAFS LC-40 coordinates
launch_lat, launch_lon = 28.562302, -80.577356

# Closest coastline point
coast_lat, coast_lon = 28.56278, -80.56789

# Calculate distance (km)
dist_km = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

# Coastline marker
folium.Marker(
    location=[coast_lat, coast_lon],
    tooltip=f"Closest coastline (~{dist_km:.2f} km)",
    icon=folium.Icon(color="green", icon="info-sign")
).add_to(site_map)

# Line connecting launch site to coastline
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [coast_lat, coast_lon]],
    weight=3,
    color="blue",
    tooltip=f"Distance to coastline: {dist_km:.2f} km"
).add_to(site_map)

site_map


In [17]:
launch_lat, launch_lon = 28.562302, -80.577356

In [18]:
# Railway
rail_lat, rail_lon = 28.56278, -80.80685

# Highway
highway_lat, highway_lon = 28.51441, -80.61493

# City (Titusville)
city_lat, city_lon = 28.60924, -80.80994

In [19]:
def add_proximity(feature_name, lat, lon, color):
    dist = calculate_distance(launch_lat, launch_lon, lat, lon)
    
    folium.Marker(
        location=[lat, lon],
        tooltip=f"{feature_name} (~{dist:.2f} km)",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(site_map)
    
    folium.PolyLine(
        locations=[[launch_lat, launch_lon], [lat, lon]],
        color=color,
        weight=3,
        tooltip=f"{feature_name} distance: {dist:.2f} km"
    ).add_to(site_map)

In [20]:
add_proximity("Railway", rail_lat, rail_lon, "purple")
add_proximity("Highway", highway_lat, highway_lon, "orange")
add_proximity("City (Titusville)", city_lat, city_lon, "red")

site_map